# Data Science Capstone Project

### This is the Shareable Jupyter Notebook for the Capstone project.

In [1]:
import pandas as pd
import numpy as np

dc = pd.read_csv(r'Data-Collisions.csv')

/home/fperez/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
(dc["ROADCOND"].isnull()==True).sum()

158

In [2]:
#dc.describe

In [1]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
